In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
import os
import shutil

In [2]:
df2 = pd.read_csv('cxr-record-list.csv')
df4 = pd.read_csv('mimic-cxr-2.0.0-chexpert.csv')
df6 = pd.read_csv('mimic-cxr-2.0.0-metadata.csv')
merge_df = df2.merge(df4,on=['subject_id','study_id'],how='inner')
merge_df = merge_df.merge(df6[['dicom_id', 'ViewPosition']],on='dicom_id',how='inner')

In [3]:
def csv_to_train_val(df,img_path,disease):
    '''
    split data to train ans validation
    disease labe must be 1 for case or 0 for control, labeled by chexpert
    images are all posterior to anterior (filtered when downloading)
    choose one PA image per subject
    '''
    file_ids = [file.replace(".jpg","") for file in os.listdir(img_path)]
    df = df[df['dicom_id'].isin(file_ids)]
    
    df.loc[:,'image_path'] = img_path+ '/' + df['dicom_id'] + '.jpg'
    
    input_df = df[df[disease].isin([0,1])].drop_duplicates('subject_id')
    train_df, val_df = train_test_split(
        input_df, 
        test_size=0.2, 
        random_state=1, 
        stratify=input_df[disease]
    )
    return train_df, val_df

In [4]:
# 1. Define the custom dataset class
class disease_Dataset(Dataset):
    def __init__(self, df, disease, transform=None):
        self.df = df
        self.transform = transform
        self.disease = disease

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['image_path']).convert("RGB") #images are gray scale, but models require RGB
        label = torch.tensor(row['Pneumonia'], dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, label


# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def generate_dataLoader(disease, train_df,val_df):
    # DataLoaders
    train_dataset = disease_Dataset(train_df,disease, transform)
    val_dataset = disease_Dataset(val_df,disease, transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    return train_loader, val_loader

In [5]:
def train(model, train_loader, val_loader, lr, weight_decay):
    # Define loss function and optimizer
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Training loop with early stopping
    num_epochs = 10
    patience = 3  # Number of epochs to wait for improvement
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_train_loss = running_loss / len(train_loader.dataset)

        # Evaluate on the validation set
        model.eval()
        val_labels = []
        val_preds = []
        val_running_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images).squeeze()
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * images.size(0)
                preds = torch.sigmoid(outputs).cpu().numpy()
                val_preds.extend(preds)
                val_labels.extend(labels.cpu().numpy())

        epoch_val_loss = val_running_loss / len(val_loader.dataset)

        # Validation metrics
        val_binary_preds = [1 if p > 0.5 else 0 for p in val_preds]
        val_accuracy = accuracy_score(val_labels, val_binary_preds)
        val_f1 = f1_score(val_labels, val_binary_preds)
        val_auc = roc_auc_score(val_labels, val_preds)

        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Training Loss: {epoch_train_loss:.4f}")
        print(f"  Validation Loss: {epoch_val_loss:.4f}")
        print(f"  Validation Accuracy: {val_accuracy:.4f}")
        print(f"  Validation F1 Score: {val_f1:.4f}")
        print(f"  Validation AUC: {val_auc:.4f}")

        # Check for early stopping
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            patience_counter = 0  # Reset counter if validation loss improves
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered. Stopping training.")
                break

    return model


In [7]:
train_df, val_df = csv_to_train_val(merge_df,'images_NEW','Pneumonia') #2700
train_loader, val_loader = generate_dataLoader('Pneumonia',train_df,val_df)

/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [40]:
%%time
# 4700 1 & all 0
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 1)
model = model.to(device)
train(model, train_loader, val_loader)

/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/20:
  Training Loss: 0.9771
  Validation Loss: 0.8893
  Validation Accuracy: 0.3025
  Validation F1 Score: 0.4378
  Validation AUC: 0.5210
Epoch 2/20:
  Training Loss: 0.8454
  Validation Loss: 0.8999
  Validation Accuracy: 0.3086
  Validation F1 Score: 0.4343
  Validation AUC: 0.5570
Epoch 3/20:
  Training Loss: 0.7504
  Validation Loss: 0.8781
  Validation Accuracy: 0.3457
  Validation F1 Score: 0.4536
  Validation AUC: 0.5857
Epoch 4/20:
  Training Loss: 0.6630
  Validation Loss: 0.8257
  Validation Accuracy: 0.4074
  Validation F1 Score: 0.4419
  Validation AUC: 0.5614
Epoch 5/20:
  Training Loss: 0.5783
  Validation Loss: 0.8266
  Validation Accuracy: 0.4321
  Validation F1 Score: 0.4321
  Validation AUC: 0.5437
Epoch 6/20:
  Training Loss: 0.5061
  Validation Loss: 0.7849
  Validation Accuracy: 0.4691
  Validation F1 Score: 0.3768
  Validation AUC: 0.5416
Epoch 7/20:
  Training Loss: 0.4278
  Validation Loss: 0.8137
  Validation Accuracy: 0.4444
  Validation F1 Score: 0.3

In [23]:
%%time
model = models.densenet121(pretrained=True)
model.classifier = nn.Linear(model.classifier.in_features, 1)
model = model.to(device)
train(model, train_loader, val_loader)

/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/20:
  Training Loss: 0.6381
  Validation Loss: 0.6231
  Validation Accuracy: 0.6916
  Validation F1 Score: 0.2353
  Validation AUC: 0.6292
Epoch 2/20:
  Training Loss: 0.5734
  Validation Loss: 0.5982
  Validation Accuracy: 0.6843
  Validation F1 Score: 0.0942
  Validation AUC: 0.6561
Epoch 3/20:
  Training Loss: 0.5273
  Validation Loss: 0.5896
  Validation Accuracy: 0.6861
  Validation F1 Score: 0.1224
  Validation AUC: 0.6706
Epoch 4/20:
  Training Loss: 0.4814
  Validation Loss: 0.5820
  Validation Accuracy: 0.6788
  Validation F1 Score: 0.1776
  Validation AUC: 0.6843
Epoch 5/20:
  Training Loss: 0.4316
  Validation Loss: 0.5822
  Validation Accuracy: 0.6861
  Validation F1 Score: 0.2110
  Validation AUC: 0.6785
Epoch 6/20:
  Training Loss: 0.3786
  Validation Loss: 0.5786
  Validation Accuracy: 0.6898
  Validation F1 Score: 0.3254
  Validation AUC: 0.6876
Epoch 7/20:
  Training Loss: 0.3127
  Validation Loss: 0.5928
  Validation Accuracy: 0.6953
  Validation F1 Score: 0.3

In [47]:
%%time 
# Define ResNet model
model_resnet = models.resnet18(pretrained=True)
model_resnet.fc = torch.nn.Linear(model_resnet.fc.in_features, 1)
model_resnet = model_resnet.to(device)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_resnet.parameters(), lr=0.0001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model_resnet.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_resnet(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_train_loss = running_loss / len(train_dataset)

    # Evaluate on the validation set
    model_resnet.eval()
    val_labels = []
    val_preds = []
    val_running_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_resnet(images).squeeze()
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            preds = torch.sigmoid(outputs).cpu().numpy()
            val_preds.extend(preds)
            val_labels.extend(labels.cpu().numpy())

    epoch_val_loss = val_running_loss / len(val_dataset)

    # Validation metrics
    val_binary_preds = [1 if p > 0.5 else 0 for p in val_preds]
    val_accuracy = accuracy_score(val_labels, val_binary_preds)
    val_f1 = f1_score(val_labels, val_binary_preds)
    val_auc = roc_auc_score(val_labels, val_preds)

    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"  Training Loss: {epoch_train_loss:.4f}")
    print(f"  Validation Loss: {epoch_val_loss:.4f}")
    print(f"  Validation Accuracy: {val_accuracy:.4f}")
    print(f"  Validation F1 Score: {val_f1:.4f}")
    print(f"  Validation AUC: {val_auc:.4f}")

/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10:
  Training Loss: 0.6382
  Validation Loss: 0.5733
  Validation Accuracy: 0.6861
  Validation F1 Score: 0.0278
  Validation AUC: 0.7130
Epoch 2/10:
  Training Loss: 0.2959
  Validation Loss: 0.6541
  Validation Accuracy: 0.7309
  Validation F1 Score: 0.2683
  Validation AUC: 0.7170
Epoch 3/10:
  Training Loss: 0.0847
  Validation Loss: 0.5407
  Validation Accuracy: 0.7534
  Validation F1 Score: 0.5045
  Validation AUC: 0.7738
Epoch 4/10:
  Training Loss: 0.0230
  Validation Loss: 0.6324
  Validation Accuracy: 0.7534
  Validation F1 Score: 0.5133
  Validation AUC: 0.7545
Epoch 5/10:
  Training Loss: 0.0078
  Validation Loss: 0.6623
  Validation Accuracy: 0.7489
  Validation F1 Score: 0.4717
  Validation AUC: 0.7755
Epoch 6/10:
  Training Loss: 0.0059
  Validation Loss: 0.6439
  Validation Accuracy: 0.7623
  Validation F1 Score: 0.5310
  Validation AUC: 0.7810
Epoch 7/10:
  Training Loss: 0.0041
  Validation Loss: 0.6519
  Validation Accuracy: 0.7534
  Validation F1 Score: 0.5

## hyper parameters tunning

In [48]:
# Hyperparameter grid
num_epochs = 10 
hyperparams = {
    'learning_rates': [1e-3, 1e-4, 1e-5],
    'batch_sizes': [32, 64],
    'weight_decays': [0, 1e-4],
}

best_hyperparams = None
best_val_auc = 0.0

for lr in hyperparams['learning_rates']:
    for batch_size in hyperparams['batch_sizes']:
        for weight_decay in hyperparams['weight_decays']:
            print(f"\nTesting: LR={lr}, Batch Size={batch_size}, Weight Decay={weight_decay}")
            
            # Reinitialize model, optimizer, and dataloaders
            model_resnet = models.resnet18(pretrained=True)
            model_resnet.fc = torch.nn.Linear(model_resnet.fc.in_features, 1)
            model_resnet = model_resnet.to(device)
            criterion = torch.nn.BCEWithLogitsLoss()
            optimizer = torch.optim.Adam(model_resnet.parameters(), lr=lr, weight_decay=weight_decay)

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size)

            # Training loop (simplified for one epoch)
            for epoch in range(num_epochs):
                model_resnet.train()
                for images, labels in train_loader:
                    images, labels = images.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model_resnet(images).squeeze()
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                # Validation evaluation
                model_resnet.eval()
                val_preds = []
                val_labels = []
                with torch.no_grad():
                    for images, labels in val_loader:
                        images, labels = images.to(device), labels.to(device)
                        outputs = model_resnet(images).squeeze()
                        val_preds.extend(torch.sigmoid(outputs).cpu().numpy())
                        val_labels.extend(labels.cpu().numpy())
                
                val_auc = roc_auc_score(val_labels, val_preds)

                # Track the best configuration
                if val_auc > best_val_auc:
                    best_val_auc = val_auc
                    best_hyperparams = {'lr': lr, 'batch_size': batch_size, 'weight_decay': weight_decay}
                    
                    
                    print(f"New best model with AUC={val_auc:.4f}")

print(f"\nBest Hyperparameters: {best_hyperparams}, Best Validation AUC: {best_val_auc:.4f}")


Testing: LR=0.001, Batch Size=32, Weight Decay=0


/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


New best model with AUC=0.6037
New best model with AUC=0.6600
New best model with AUC=0.6921
New best model with AUC=0.7130

Testing: LR=0.001, Batch Size=32, Weight Decay=0.0001


/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


New best model with AUC=0.7140
New best model with AUC=0.7393

Testing: LR=0.001, Batch Size=64, Weight Decay=0


...

New best model with AUC=0.7790
New best model with AUC=0.7880
New best model with AUC=0.7928

Testing: LR=1e-05, Batch Size=64, Weight Decay=0


/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Testing: LR=1e-05, Batch Size=64, Weight Decay=0.0001


/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Best Hyperparameters: {'lr': 1e-05, 'batch_size': 32, 'weight_decay': 0.0001}, Best Validation AUC: 0.7928


In [24]:
# try learning rate and weight dacay

for lr in [1e-3, 1e-4, 1e-5]:
    for weight_decay in [0, 1e-4]:
        model = models.resnet18(pretrained=True)
        model.fc = torch.nn.Linear(model.fc.in_features, 1)
        model = model.to(device)
        print(f"=====lr: {lr}, decay: {weight_decay}=====")
        train(model, train_loader, val_loader, lr, weight_decay)

/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


=====lr: 0.001, decay: 0=====
Epoch 1/10:
  Training Loss: 0.6479
  Validation Loss: 0.5885
  Validation Accuracy: 0.6837
  Validation F1 Score: 0.4335
  Validation AUC: 0.6721
Epoch 2/10:
  Training Loss: 0.5862
  Validation Loss: 0.6640
  Validation Accuracy: 0.6964
  Validation F1 Score: 0.0654
  Validation AUC: 0.6549
Epoch 3/10:
  Training Loss: 0.5766
  Validation Loss: 0.6036
  Validation Accuracy: 0.6964
  Validation F1 Score: 0.3470
  Validation AUC: 0.6305
Epoch 4/10:
  Training Loss: 0.5580
  Validation Loss: 0.6936
  Validation Accuracy: 0.6263
  Validation F1 Score: 0.4172
  Validation AUC: 0.6166
Early stopping triggered. Stopping training.


/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


=====lr: 0.001, decay: 0.0001=====
Epoch 1/10:
  Training Loss: 0.6562
  Validation Loss: 0.6730
  Validation Accuracy: 0.6900
  Validation F1 Score: 0.0000
  Validation AUC: 0.6546
Epoch 2/10:
  Training Loss: 0.5940
  Validation Loss: 0.5925
  Validation Accuracy: 0.6943
  Validation F1 Score: 0.0270
  Validation AUC: 0.6361
Epoch 3/10:
  Training Loss: 0.5759
  Validation Loss: 0.6244
  Validation Accuracy: 0.6794
  Validation F1 Score: 0.2775
  Validation AUC: 0.6198
Epoch 4/10:
  Training Loss: 0.5564
  Validation Loss: 0.6064
  Validation Accuracy: 0.6964
  Validation F1 Score: 0.1006
  Validation AUC: 0.6587
Epoch 5/10:
  Training Loss: 0.5300
  Validation Loss: 0.6357
  Validation Accuracy: 0.6964
  Validation F1 Score: 0.1538
  Validation AUC: 0.6659
Early stopping triggered. Stopping training.


/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


...

=====lr: 0.0001, decay: 0.0001=====
Epoch 1/10:
  Training Loss: 0.6116
  Validation Loss: 0.5780
  Validation Accuracy: 0.6858
  Validation F1 Score: 0.3934
  Validation AUC: 0.6989
Epoch 2/10:
  Training Loss: 0.3569
  Validation Loss: 0.8998
  Validation Accuracy: 0.5605
  Validation F1 Score: 0.4733
  Validation AUC: 0.6241
Epoch 3/10:
  Training Loss: 0.1057
  Validation Loss: 0.8787
  Validation Accuracy: 0.6242
  Validation F1 Score: 0.3218
  Validation AUC: 0.6184
Epoch 4/10:
  Training Loss: 0.0371
  Validation Loss: 0.9318
  Validation Accuracy: 0.6561
  Validation F1 Score: 0.4214
  Validation AUC: 0.6525
Early stopping triggered. Stopping training.


/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


=====lr: 1e-05, decay: 0=====
Epoch 1/10:
  Training Loss: 0.6331
  Validation Loss: 0.6678
  Validation Accuracy: 0.5732
  Validation F1 Score: 0.3964
  Validation AUC: 0.5753
Epoch 2/10:
  Training Loss: 0.5443
  Validation Loss: 0.5810
  Validation Accuracy: 0.6900
  Validation F1 Score: 0.2474
  Validation AUC: 0.6737
Epoch 3/10:
  Training Loss: 0.4797
  Validation Loss: 0.5750
  Validation Accuracy: 0.6943
  Validation F1 Score: 0.3514
  Validation AUC: 0.6871
Epoch 4/10:
  Training Loss: 0.4165
  Validation Loss: 0.5719
  Validation Accuracy: 0.6964
  Validation F1 Score: 0.3529
  Validation AUC: 0.6894
Epoch 5/10:
  Training Loss: 0.3515
  Validation Loss: 0.5710
  Validation Accuracy: 0.6985
  Validation F1 Score: 0.4034
  Validation AUC: 0.6972
Epoch 6/10:
  Training Loss: 0.2837
  Validation Loss: 0.5739
  Validation Accuracy: 0.7028
  Validation F1 Score: 0.3966
  Validation AUC: 0.6991
Epoch 7/10:
  Training Loss: 0.2222
  Validation Loss: 0.5829
  Validation Accuracy: 0.6

/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


=====lr: 1e-05, decay: 0.0001=====
Epoch 1/10:
  Training Loss: 0.6431
  Validation Loss: 0.6199
  Validation Accuracy: 0.6794
  Validation F1 Score: 0.2775
  Validation AUC: 0.6321
Epoch 2/10:
  Training Loss: 0.5512
  Validation Loss: 0.5753
  Validation Accuracy: 0.7325
  Validation F1 Score: 0.3762
  Validation AUC: 0.6915
Epoch 3/10:
  Training Loss: 0.4813
  Validation Loss: 0.5634
  Validation Accuracy: 0.7282
  Validation F1 Score: 0.3786
  Validation AUC: 0.7056
Epoch 4/10:
  Training Loss: 0.4186
  Validation Loss: 0.5555
  Validation Accuracy: 0.7197
  Validation F1 Score: 0.3889
  Validation AUC: 0.7154
Epoch 5/10:
  Training Loss: 0.3514
  Validation Loss: 0.5537
  Validation Accuracy: 0.7346
  Validation F1 Score: 0.4541
  Validation AUC: 0.7179
Epoch 6/10:
  Training Loss: 0.2820
  Validation Loss: 0.5543
  Validation Accuracy: 0.7346
  Validation F1 Score: 0.4541
  Validation AUC: 0.7172
Epoch 7/10:
  Training Loss: 0.2240
  Validation Loss: 0.5661
  Validation Accuracy

In [29]:
def train(model, train_loader, val_loader, lr=1e-5, weight_decay=1e-4, dropout_prob=0.5, use_scheduler=True, augmentations=None):

    # Apply augmentations if provided
    if augmentations:
        train_loader.dataset.transform = augmentations

    # Add dropout to the fully connected layer dynamically
    model.fc = torch.nn.Sequential(
        torch.nn.Dropout(p=dropout_prob),
        torch.nn.Linear(model.fc.in_features, 1)
    ).to(device)  # Ensure the modified layer is on the correct device

    # Define loss function and optimizer
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Optionally use a learning rate scheduler
    scheduler = None
    if use_scheduler:
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.1, patience=2, verbose=True
        )

    # Training loop with early stopping
    num_epochs = 10
    patience = 3  # Number of epochs to wait for improvement
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            # Move images and labels to the specified device
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_train_loss = running_loss / len(train_loader.dataset)

        # Evaluate on the validation set
        model.eval()
        val_labels = []
        val_preds = []
        val_running_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                # Move images and labels to the specified device
                images, labels = images.to(device), labels.to(device)
                outputs = model(images).squeeze()
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * images.size(0)
                preds = torch.sigmoid(outputs).cpu().numpy()  # Move predictions to CPU for evaluation
                val_preds.extend(preds)
                val_labels.extend(labels.cpu().numpy())  # Move labels to CPU for evaluation

        epoch_val_loss = val_running_loss / len(val_loader.dataset)

        # Validation metrics
        val_binary_preds = [1 if p > 0.5 else 0 for p in val_preds]
        val_accuracy = accuracy_score(val_labels, val_binary_preds)
        val_f1 = f1_score(val_labels, val_binary_preds)
        val_auc = roc_auc_score(val_labels, val_preds)

        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Training Loss: {epoch_train_loss:.4f}")
        print(f"  Validation Loss: {epoch_val_loss:.4f}")
        print(f"  Validation Accuracy: {val_accuracy:.4f}")
        print(f"  Validation F1 Score: {val_f1:.4f}")
        print(f"  Validation AUC: {val_auc:.4f}")

        # Update the learning rate scheduler
        if use_scheduler:
            scheduler.step(epoch_val_loss)

        # Check for early stopping
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            patience_counter = 0  # Reset counter if validation loss improves
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered. Stopping training.")
                break

    return model


In [ ]:
from torchvision import transforms

# Define augmentations
basic_augmentations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

no_augmentations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# try augmentations, use_scheduler, dropout_prob
for dropout_prob in [0.3, 0.5]:
    for augmentations in [no_augmentations, basic_augmentations]:
        for use_scheduler in [True, False]:
            model = models.resnet18(pretrained=True)
            model.fc = torch.nn.Linear(model.fc.in_features, 1)
            model = model.to(device)

            print(f"=====lr: {lr}, decay: {weight_decay}, dropout: {dropout_prob}, scheduler: {use_scheduler}, augmentations: {augmentations is not None}=====")
            train(model, train_loader, val_loader, lr, weight_decay, dropout_prob, use_scheduler, augmentations)


=====lr: 1e-05, decay: 0.0001, dropout: 0.3, scheduler: True, augmentations: True=====
Epoch 1/10:
  Training Loss: 0.6602
  Validation Loss: 0.6042
  Validation Accuracy: 0.6858
  Validation F1 Score: 0.0864
  Validation AUC: 0.6128
Epoch 2/10:
  Training Loss: 0.5814
  Validation Loss: 0.5913
  Validation Accuracy: 0.6879
  Validation F1 Score: 0.3408
  Validation AUC: 0.6699
Epoch 3/10:
  Training Loss: 0.5214
  Validation Loss: 0.5816
  Validation Accuracy: 0.6921
  Validation F1 Score: 0.3498
  Validation AUC: 0.6796
Epoch 4/10:
  Training Loss: 0.4651
  Validation Loss: 0.5717
  Validation Accuracy: 0.7070
  Validation F1 Score: 0.3839
  Validation AUC: 0.6937
Epoch 5/10:
  Training Loss: 0.4134
  Validation Loss: 0.5701
  Validation Accuracy: 0.7028
  Validation F1 Score: 0.3805
  Validation AUC: 0.6957
Epoch 6/10:
  Training Loss: 0.3533
  Validation Loss: 0.5745
  Validation Accuracy: 0.6943
  Validation F1 Score: 0.3739
  Validation AUC: 0.6950
Epoch 7/10:
  Training Loss: 0.

/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10:
  Training Loss: 0.7568
  Validation Loss: 0.7303
  Validation Accuracy: 0.4671
  Validation F1 Score: 0.4803
  Validation AUC: 0.6101
Epoch 2/10:
  Training Loss: 0.6377
  Validation Loss: 0.6509
  Validation Accuracy: 0.6093
  Validation F1 Score: 0.4945
  Validation AUC: 0.6465
Epoch 3/10:
  Training Loss: 0.5570
  Validation Loss: 0.6144
  Validation Accuracy: 0.6688
  Validation F1 Score: 0.4429
  Validation AUC: 0.6649
Epoch 4/10:
  Training Loss: 0.4867
  Validation Loss: 0.6015
  Validation Accuracy: 0.6879
  Validation F1 Score: 0.4535
  Validation AUC: 0.6695
Epoch 5/10:
  Training Loss: 0.4216
  Validation Loss: 0.5849
  Validation Accuracy: 0.7091
  Validation F1 Score: 0.4498
  Validation AUC: 0.6810
Epoch 6/10:
  Training Loss: 0.3636
  Validation Loss: 0.5917
  Validation Accuracy: 0.6921
  Validation F1 Score: 0.4689
  Validation AUC: 0.6837
Epoch 7/10:
  Training Loss: 0.2949
  Validation Loss: 0.5837
  Validation Accuracy: 0.7261
  Validation F1 Score: 0.4

/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/hsieh.fe/.local/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/10:
  Training Loss: 0.6791
  Validation Loss: 0.6387
  Validation Accuracy: 0.6454
  Validation F1 Score: 0.2304
  Validation AUC: 0.5650
Epoch 2/10:
  Training Loss: 0.6329
  Validation Loss: 0.6015
  Validation Accuracy: 0.7049
  Validation F1 Score: 0.3349
  Validation AUC: 0.6514
Epoch 3/10:
  Training Loss: 0.6034
  Validation Loss: 0.5901
  Validation Accuracy: 0.7113
  Validation F1 Score: 0.3704
  Validation AUC: 0.6688
Epoch 4/10:
  Training Loss: 0.5981
  Validation Loss: 0.5778
  Validation Accuracy: 0.7070
  Validation F1 Score: 0.3491
  Validation AUC: 0.6870
Epoch 5/10:
  Training Loss: 0.5770
  Validation Loss: 0.5691
  Validation Accuracy: 0.7134
  Validation F1 Score: 0.3147
  Validation AUC: 0.6949
Epoch 6/10:
  Training Loss: 0.5714
  Validation Loss: 0.5660
  Validation Accuracy: 0.7176
  Validation F1 Score: 0.3512
  Validation AUC: 0.6973
Epoch 7/10:
  Training Loss: 0.5516
  Validation Loss: 0.5649
  Validation Accuracy: 0.7219
  Validation F1 Score: 0.3

...

Epoch 1/10:
  Training Loss: 0.6960
  Validation Loss: 0.6745
  Validation Accuracy: 0.5563
  Validation F1 Score: 0.4367
  Validation AUC: 0.5941
Epoch 2/10:
  Training Loss: 0.6081
  Validation Loss: 0.5965
  Validation Accuracy: 0.7049
  Validation F1 Score: 0.3085
  Validation AUC: 0.6435
Epoch 3/10:
  Training Loss: 0.5528
  Validation Loss: 0.5825
  Validation Accuracy: 0.7091
  Validation F1 Score: 0.3184
  Validation AUC: 0.6675
Epoch 4/10:
  Training Loss: 0.5293
  Validation Loss: 0.5735
  Validation Accuracy: 0.7070
  Validation F1 Score: 0.3030
  Validation AUC: 0.6813
Epoch 5/10:
  Training Loss: 0.4696
  Validation Loss: 0.5702
  Validation Accuracy: 0.7091
  Validation F1 Score: 0.3251
  Validation AUC: 0.6857
Epoch 6/10:
  Training Loss: 0.4429
  Validation Loss: 0.5696
  Validation Accuracy: 0.7155
  Validation F1 Score: 0.3909
  Validation AUC: 0.6901
Epoch 7/10:
  Training Loss: 0.3789
  Validation Loss: 0.5720
  Validation Accuracy: 0.7155
  Validation F1 Score: 0.4

/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/hsieh.fe/.local/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/10:
  Training Loss: 0.8041
  Validation Loss: 0.8183
  Validation Accuracy: 0.3355
  Validation F1 Score: 0.4686
  Validation AUC: 0.5363
Epoch 2/10:
  Training Loss: 0.7239
  Validation Loss: 0.6994
  Validation Accuracy: 0.5265
  Validation F1 Score: 0.4989
  Validation AUC: 0.6163
Epoch 3/10:
  Training Loss: 0.6788
  Validation Loss: 0.6456
  Validation Accuracy: 0.6200
  Validation F1 Score: 0.4751
  Validation AUC: 0.6588
Epoch 4/10:
  Training Loss: 0.6514
  Validation Loss: 0.6246
  Validation Accuracy: 0.6561
  Validation F1 Score: 0.4706
  Validation AUC: 0.6802
Epoch 5/10:
  Training Loss: 0.6276
  Validation Loss: 0.5900
  Validation Accuracy: 0.7113
  Validation F1 Score: 0.4380
  Validation AUC: 0.6907
Epoch 6/10:
  Training Loss: 0.6054
  Validation Loss: 0.5818
  Validation Accuracy: 0.7028
  Validation F1 Score: 0.4167
  Validation AUC: 0.6903
Epoch 7/10:
  Training Loss: 0.5753
  Validation Loss: 0.5679
  Validation Accuracy: 0.7304
  Validation F1 Score: 0.3

/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10:
  Training Loss: 0.7531
  Validation Loss: 0.6340
  Validation Accuracy: 0.6815
  Validation F1 Score: 0.1071
  Validation AUC: 0.5180
Epoch 2/10:
  Training Loss: 0.6999
  Validation Loss: 0.6348
  Validation Accuracy: 0.6539
  Validation F1 Score: 0.3064
  Validation AUC: 0.5929
Epoch 3/10:
  Training Loss: 0.6484
  Validation Loss: 0.6167
  Validation Accuracy: 0.6773
  Validation F1 Score: 0.3153
  Validation AUC: 0.6140
Epoch 4/10:
  Training Loss: 0.6350
  Validation Loss: 0.6006
  Validation Accuracy: 0.6985
  Validation F1 Score: 0.3039
  Validation AUC: 0.6298
Epoch 5/10:
  Training Loss: 0.6101
  Validation Loss: 0.5971
  Validation Accuracy: 0.6900
  Validation F1 Score: 0.2700
  Validation AUC: 0.6356
Epoch 6/10:
  Training Loss: 0.6031
  Validation Loss: 0.5900
  Validation Accuracy: 0.7006
  Validation F1 Score: 0.2985
  Validation AUC: 0.6485
Epoch 7/10:
  Training Loss: 0.5806
  Validation Loss: 0.5863
  Validation Accuracy: 0.7134
  Validation F1 Score: 0.3

In [ ]:
%%time
#2700
model_vgg16 = models.vgg16(pretrained=True)
model_vgg16.classifier[6] = torch.nn.Linear(model_vgg16.classifier[6].in_features, 1)
model_vgg16 = model_vgg16.to(device)

train(model_vgg16,train_loader,val_loader,1e-5,1e-4)



/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
%%time
model_resnet50 = models.resnet50(pretrained=True)
model_resnet50.fc = torch.nn.Linear(model_resnet50.fc.in_features, 1)
model_resnet50 = model_resnet50.to(device)
train(model_resnet50, train_loader, val_loader)

Epoch 1/20:
  Training Loss: 0.7524
  Validation Loss: 0.7403
  Validation Accuracy: 0.3632
  Validation F1 Score: 0.4772
  Validation AUC: 0.5192
Epoch 2/20:
  Training Loss: 0.6062
  Validation Loss: 0.6731
  Validation Accuracy: 0.5812
  Validation F1 Score: 0.3467
  Validation AUC: 0.5837
Epoch 3/20:
  Training Loss: 0.4913
  Validation Loss: 0.6254
  Validation Accuracy: 0.6538
  Validation F1 Score: 0.4000
  Validation AUC: 0.6439
Epoch 4/20:
  Training Loss: 0.3666
  Validation Loss: 0.6126
  Validation Accuracy: 0.6752
  Validation F1 Score: 0.4154
  Validation AUC: 0.6478
Epoch 5/20:
  Training Loss: 0.2379
  Validation Loss: 0.6077
  Validation Accuracy: 0.6838
  Validation F1 Score: 0.4127
  Validation AUC: 0.6740
Epoch 6/20:
  Training Loss: 0.1357
  Validation Loss: 0.6404
  Validation Accuracy: 0.6838
  Validation F1 Score: 0.4127
  Validation AUC: 0.6597
Epoch 7/20:
  Training Loss: 0.0782
  Validation Loss: 0.6699
  Validation Accuracy: 0.6752
  Validation F1 Score: 0.3

KeyboardInterrupt: 

In [66]:
model_resnet50_unfreeze_last3 = models.resnet50(pretrained=True)

# Replace the fully connected (fc) layer with your custom output layer
model_resnet50_unfreeze_last3.fc = nn.Linear(model_resnet50_unfreeze_last3.fc.in_features, 1)

# Freeze all layers except the last three
for name, param in model_resnet50_unfreeze_last3.named_parameters():
    param.requires_grad = False  # Freeze all layers

# Unfreeze the last three layers (you may adjust depending on the layer names)
for name, param in model_resnet50_unfreeze_last3.named_parameters():
    if "layer4" in name or "fc" in name:  # Modify according to ResNet layer structure
        param.requires_grad = True

# Move model to the device
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

# DataLoaders
train_dataset = PneumoniaDataset(train_df, transform)
val_dataset = PneumoniaDataset(val_df, transform)
#test_dataset = PneumoniaDataset(test_df, transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
model_resnet50_unfreeze_last3 = model_resnet50_unfreeze_last3.to(device)
train(model_resnet50_unfreeze_last3,train_loader,val_loader)

Epoch 1/20:
  Training Loss: 0.6694
  Validation Loss: 0.6297
  Validation Accuracy: 0.6861
  Validation F1 Score: 0.0000
  Validation AUC: 0.5891
Epoch 2/20:
  Training Loss: 0.5929
  Validation Loss: 0.6126
  Validation Accuracy: 0.6816
  Validation F1 Score: 0.0000
  Validation AUC: 0.6218
Epoch 3/20:
  Training Loss: 0.5358
  Validation Loss: 0.6018
  Validation Accuracy: 0.6861
  Validation F1 Score: 0.0789
  Validation AUC: 0.6409
Epoch 4/20:
  Training Loss: 0.4627
  Validation Loss: 0.5883
  Validation Accuracy: 0.6906
  Validation F1 Score: 0.1882
  Validation AUC: 0.6563
Epoch 5/20:
  Training Loss: 0.3654
  Validation Loss: 0.5924
  Validation Accuracy: 0.6906
  Validation F1 Score: 0.2887
  Validation AUC: 0.6588
Epoch 6/20:
  Training Loss: 0.2542
  Validation Loss: 0.6177
  Validation Accuracy: 0.6682
  Validation F1 Score: 0.3509
  Validation AUC: 0.6483
Epoch 7/20:
  Training Loss: 0.1620
  Validation Loss: 0.6571
  Validation Accuracy: 0.6278
  Validation F1 Score: 0.3

KeyboardInterrupt: 

In [15]:
train_loader.dataset